In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import ukbb_recessive.regression.plotting as plotting

In [ ]:
# plps file
plps = pd.read_csv(
    ".../450k/plp_selection/basic/new_gene_names/"
    "new_freq/new_relatedness/all_chr_total_presumable_plps_HFE_final_sorted.txt", sep="\t")

#gene panel
gene_panel = pd.read_csv(".../450k/regions/gene-panel-gencode-v34.txt", header=None)
gene_panel.columns = ['gene', 'panel']

# samples of interest: European & non-related without hom and comp_het
# european_non_rel_samples = (
#     ".../450k/samples/european_non_related_no_withdrawal_to_include_450k.no_hom_comp_het.txt"
# )
european_non_rel_samples = (
    ".../450k/samples/european_non_related_no_withdrawal_to_include_450k.txt"
)

with open(european_non_rel_samples, 'r') as f:
    european_non_rel_samples = [l.strip() for l in f.readlines()]

N = len(european_non_rel_samples)
print (f"Number of samples: {N}")

# mild plps; were taken from Hila's excel file; they are the mild variants
mild_plps = pd.read_csv('mild_variants.csv', sep='\t').rename(columns={'hets': 'mild_hets', 'homs': 'mild_homs'}).drop('panel', axis=1)


In [ ]:
# separate mild variants
plps = plps.merge(mild_plps, how='left')
plps[['mild_hets', 'mild_homs']] = plps[['mild_hets', 'mild_homs']].fillna(0)
plps['hets'] = plps['hets'] - plps['mild_hets']
plps['homs'] = plps['homs'] - plps['mild_homs']

# add gene panel information
plps = plps.merge(gene_panel, how='inner', on='gene')
plps = plps[~plps['panel'].isin(['Tumor', 'No_panel', 'Cardiovascular'])]

In [ ]:
def add_data_per_gene(plps):
    # per gene genotypes count
    plps_gene = plps.groupby(['panel', 'gene'])[['hets', 'homs', 'mild_hets', 'mild_homs']].sum().reset_index()

    # allele cound
    plps_gene['AC'] = plps_gene['hets'] + 2*plps_gene['homs']
    plps_gene['mild_AC'] = plps_gene['mild_hets'] + 2*plps_gene['mild_homs']

    # allele frequency
    plps_gene['AF'] = plps_gene['AC'] / (2*N)
    plps_gene['mild_AF'] = plps_gene['mild_AC'] / (2*N)
    plps_gene['total_AF'] = plps_gene['AF'] + plps_gene['mild_AF']

    # heterozygotes probability
    plps_gene['hets_expected'] = 2*plps_gene['AF']*(1-plps_gene['AF'])
    plps_gene['mild_hets_expected'] = 2*plps_gene['mild_AF']*(1-plps_gene['mild_AF'])

    # at-risk-couples
    plps_gene['at_risk_couples'] = (plps_gene['hets_expected']**2 + 2*plps_gene['hets_expected']*plps_gene['mild_hets_expected']) 
    plps_gene['at_risk_couples_first_cousins'] = plps_gene['hets_expected']*0.125

    plps_gene['CR'] = plps_gene['at_risk_couples_first_cousins'] / plps_gene['at_risk_couples'] 

    return plps_gene

def add_data_per_panel(plps_gene):
    # calculate CR on a panel level
    plps_panel = plps_gene.groupby('panel').agg({'at_risk_couples': 'sum', 'at_risk_couples_first_cousins': 'sum', 'total_AF': 'std'}).reset_index()

    plps_panel['affected_offsprings']  = plps_panel['at_risk_couples'] * 100000 / 4
    plps_panel['affected_offsprings_first_cousins']  = plps_panel['at_risk_couples_first_cousins'] * 100000 / 4

    plps_panel['CR'] = plps_panel['at_risk_couples_first_cousins'] / plps_panel['at_risk_couples']

    return plps_panel

plps_gene = add_data_per_gene(plps)
plps_panel = add_data_per_panel(plps_gene)
plps_panel = plps_panel.sort_values('CR', ascending=False).reset_index(drop=True)

plps_panel.to_csv('../../../../data/tables/CR.csv', sep='\t', index=False)

In [ ]:
results = []

for threshold in range(1, 5350, 10):
    plps_rare = plps[(plps['hets'] <= threshold)].copy()
    plps_gene = add_data_per_gene(plps_rare)
    plps_panel = add_data_per_panel(plps_gene)

    plps_panel = plps_panel.sort_values('CR', ascending=False).reset_index(drop=True)
    plps_panel['rank'] = plps_panel.index + 1
    plps_panel['threshold'] = threshold
    results.append(plps_panel)

results = pd.concat(results)


In [ ]:
# sns.scatterplot(data=results[results['panel'].isin(['ID-total', 'Neuromuscular'])], x='threshold', y='rank', hue='panel')

sns.scatterplot(data=results[results['panel'].isin(['ID-total'])], x='threshold', y='rank', hue='panel')

In [ ]:
sns.scatterplot(data=results[results['panel'].isin([ 'Neuromuscular'])], x='threshold', y='rank', hue='panel')


In [ ]:
results[results['threshold'] == 1051]